In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from datetime import datetime
from itertools import islice

# Import API key
from config import ALEX_API

In [23]:
fl_census_data = pd.read_csv('Resources/ACSST5Y2019.S1901_data_with_overlays_2021-05-24T165952.csv', header=1)
fl_codes = []

for i,j in fl_census_data.iterrows():
    fl_codes.append(j['Geographic Area Name'][6:11])


split_codes = [iter(fl_codes)] * 123
    
zip_codes = list(zip(*split_codes))

# zip_codes[0]

In [48]:
count = 0
for code in zip_codes[2]:
    count += 1

print(count)
zip_codes[3]

123


('32927',
 '32931',
 '32934',
 '32935',
 '32937',
 '32940',
 '32948',
 '32949',
 '32950',
 '32951',
 '32952',
 '32953',
 '32955',
 '32958',
 '32960',
 '32962',
 '32963',
 '32966',
 '32967',
 '32968',
 '32970',
 '32976',
 '33001',
 '33004',
 '33009',
 '33010',
 '33012',
 '33013',
 '33014',
 '33015',
 '33016',
 '33018',
 '33019',
 '33020',
 '33021',
 '33023',
 '33024',
 '33025',
 '33026',
 '33027',
 '33028',
 '33029',
 '33030',
 '33031',
 '33032',
 '33033',
 '33034',
 '33035',
 '33036',
 '33037',
 '33039',
 '33040',
 '33042',
 '33043',
 '33050',
 '33051',
 '33054',
 '33055',
 '33056',
 '33060',
 '33062',
 '33063',
 '33064',
 '33065',
 '33066',
 '33067',
 '33068',
 '33069',
 '33070',
 '33071',
 '33073',
 '33076',
 '33101',
 '33109',
 '33122',
 '33125',
 '33126',
 '33127',
 '33128',
 '33129',
 '33130',
 '33131',
 '33132',
 '33133',
 '33134',
 '33135',
 '33136',
 '33137',
 '33138',
 '33139',
 '33140',
 '33141',
 '33142',
 '33143',
 '33144',
 '33145',
 '33146',
 '33147',
 '33149',
 '33150',


In [104]:
# Read in historical mortgage rate
mortgage_data = pd.read_csv('Resources/morgage_rates.csv')
year_mortgage = []
month_mortgage = []
mortgage_rate = []

# Loop to limit data for 2019 & 2020
for i,j in mortgage_data.iterrows():
    if j['DATE']>='2019%' and j['DATE']<'2021%':
        year_mortgage.append(pd.to_datetime(j['DATE']).year)
        month_mortgage.append(pd.to_datetime(j['DATE']).month)
        mortgage_rate.append(j['MORTGAGE30US'])


# Create mortgage rate DF grouped by year and month to get the mean
mortgage_rates = pd.DataFrame({"year":year_mortgage, "month":month_mortgage, 
                        "mortgage_rate":mortgage_rate}).groupby(['year', 'month'])

# Export csv of calculated monthly rates
mortgage_rates.mean().to_csv('Resources/monthly_morgage_rates.csv')

In [50]:
# Establish API URL
property_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/"

sales_trend = "salestrend/snapshot?geoid=ZI"

sales_trend_dates = "&interval=monthly&startyear=2019&endyear=2020&startmonth=january&endmonth=december"

headers = {'Accept': 'application/json', 'apikey': ALEX_API}


# Establish variable for information desired
year = []
month = []
zip_code_sales = []
total_sales = []
average_price = []
median_price = []
sales_no_data = []

for code in zip_codes[3]:
    # Make API call
    sales_response = requests.post( f'{property_url}{sales_trend}{code}{sales_trend_dates}',
                                    headers = headers)
    
    if sales_response.status_code != 200:
        sales_no_data.append(sales_response.url)
    else:
        sales = sales_response.json()

        # Loop through API data and append to variables
        for i in sales['salestrends']:
            year.append(pd.to_datetime(i['daterange']['start']).year)
            month.append(pd.to_datetime(i['daterange']['start']).month)
            zip_code_sales.append(i['location']['geoID'][2:7])
            total_sales.append(i['SalesTrend']['homesalecount'])
            average_price.append(i['SalesTrend']['avgsaleprice'])
            median_price.append(i['SalesTrend']['medsaleprice'])
            
        # Set delay based on API call limitations
        time.sleep(7)

# Create new DF with data
sales_data = pd.DataFrame({"zipcode": zip_code_sales, "year":year, "month":month, 
                        "total_sales":total_sales, "avg_sale_price":average_price,
                        "med_sale_price":median_price})

sales_data

,zipcode,year,month,total_sales,avg_sale_price,med_sale_price
0,32927,2019,1,36,145583.0,155500.0
1,32927,2019,2,37,175495.0,160000.0
2,32927,2019,3,43,175881.0,178000.0
3,32927,2019,4,60,174315.0,177500.0
4,32927,2019,5,39,180377.0,168000.0
...,...,...,...,...,...,...
2865,33180,2020,8,80,442480.0,365000.0
2866,33180,2020,9,90,441617.0,345000.0
2867,33180,2020,10,100,417141.0,332500.0
2868,33180,2020,11,87,451337.0,330400.0


In [51]:
rates = pd.read_csv('Resources/monthly_morgage_rates.csv')

final_sales = pd.merge(sales_data, rates, on=['year','month'])

final_sales

,zipcode,year,month,total_sales,avg_sale_price,med_sale_price,mortgage_rate
0,32927,2019,1,36,145583.0,155500.0,4.464
1,32931,2019,1,18,301744.0,302500.0,4.464
2,32934,2019,1,20,244859.0,223500.0,4.464
3,32935,2019,1,53,159411.0,151900.0,4.464
4,32937,2019,1,55,353242.0,280000.0,4.464
...,...,...,...,...,...,...,...
2865,33176,2020,12,102,507554.0,355000.0,2.684
2866,33177,2020,12,61,320751.0,320000.0,2.684
2867,33178,2020,12,147,373714.0,355000.0,2.684
2868,33179,2020,12,73,343797.0,298000.0,2.684


In [52]:
final_sales.to_csv('Resources/Sales_Area_Data/sales_call_4.csv')

In [53]:
# Establish API URL
community_url = "https://api.gateway.attomdata.com/communityapi/v2.0.0"

area_url = "/area/full?AreaId=ZI"


# Establish variable for information desired
zip_code_area = []
prop_tax = []
owner_occupied = []
renter_occupied = []
total_vacant =[]
total_dwellings = []
home_age = []
studio_rent = []
one_bed_rent = []
two_bed_rent = []
three_bed_rent = []
four_bed_rent = []
fte_employed = []
unemployed = []
avg_income = []
proj_income = []
expense_index = []
avg_commute = []
crime_index = []
area_no_data = []

for code in zip_codes[3]:
    
    # Make API call
    area_response = requests.post( f'{community_url}{area_url}{code}', headers = headers)
    if area_response.status_code != 200:
        area_no_data.append(area_response.url)
    else:
        area = area_response.json()
        for i in area:
            zip_code_area.append(area['response']['inputparameter']['AreaId'][2:7])
        for i in area['response']['result']['package']['item']:
            prop_tax.append(i['avg_prop_tax'])
            owner_occupied.append(i['dwlowned'])
            renter_occupied.append(i['dwlrent'])
            total_vacant.append(i['dwlvacnt'])
            total_dwellings.append(i['dwltotal'])
            home_age.append(i['houmedage'])
            studio_rent.append(i['studio_county'])
            one_bed_rent.append(i['one_bed_county'])
            two_bed_rent.append(i['two_bed_county'])
            three_bed_rent.append(i['three_bed_county'])
            four_bed_rent.append(i['four_bed_county'])
            fte_employed.append(i['emptotal'])
            unemployed.append(i['empunemp'])
            avg_income.append(i['inccyavehh'])
            proj_income.append(i['incavehhpy_5'])
            expense_index.append(i['idxexptotal'])
            avg_commute.append(i['trwave'])
            crime_index.append(i['crmcytotc'])
            


    # Set delay based on API call limitations
    time.sleep(7)

area_data = pd.DataFrame({'zipcode': zip_code_area, 'property_tax': prop_tax,
                        'owner_occupied': owner_occupied, 'renter_occupied': renter_occupied,
                        'total_vacant': total_vacant, 'total_dwellings': total_dwellings,
                        'studio_rent': studio_rent, 'one_bed_rent': one_bed_rent,
                        'two_bed_rent': two_bed_rent, 'three_bed_rent': three_bed_rent,
                        'four_bed_rent': four_bed_rent, 'fte_employed': fte_employed,
                        'unemployed':unemployed, 'average_income':avg_income,
                        'projected_income':proj_income, 'expense_index':expense_index,
                        'average_commute':avg_commute, 'crime_index':crime_index})
area_data

,zipcode,property_tax,owner_occupied,renter_occupied,total_vacant,total_dwellings,studio_rent,one_bed_rent,two_bed_rent,three_bed_rent,four_bed_rent,fte_employed,unemployed,average_income,projected_income,expense_index,average_commute,crime_index
0,32927,1588,8709,1594,1498,11800,759,921,1134,1525,1826,2736,1183,72750,80917,98,27,116
1,32931,3448,4962,2325,3211,10498,759,921,1134,1525,1826,8335,404,89855,99941,74,23,210
2,32934,3104,6122,1612,696,8430,759,921,1134,1525,1826,8470,374,95313,106275,95,22,123
3,32935,1619,12244,6681,2343,21268,759,921,1134,1525,1826,17330,1433,63068,70159,80,21,166
4,32937,3440,8813,2894,2361,14069,759,921,1134,1525,1826,5733,793,99672,110624,88,23,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,33175,3702,12960,3237,467,16664,1057,1231,1551,2068,2483,10897,2369,78734,93292,134,34,572
112,33176,4246,13462,5214,1210,19886,1057,1231,1551,2068,2483,29139,2574,113539,134437,117,30,517
113,33177,3182,11920,3780,802,16502,1057,1231,1551,2068,2483,5448,2765,71661,85036,137,35,307
114,33179,2428,12077,5569,2109,19755,1057,1231,1551,2068,2483,12024,3374,68685,80997,98,29,424


In [54]:
area_data.to_csv('Resources/Sales_Area_Data/area_call_4.csv')